In [36]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html

class SMA_Crossup_50_200(bt.Indicator):
    lines = ("sma_up_50_200",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=200)
        self.lines.sma_up_50_200 = bt.indicators.CrossUp(self.sma_short, self.sma_long)
        
class SMA_Crossdown_50_200(bt.Indicator):
    lines = ("sma_down_50_200",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=200)
        self.lines.sma_down_50_200 = bt.indicators.CrossDown(self.sma_short, self.sma_long)

class SMA_Crossup_5_30(bt.Indicator):
    lines = ("sma_up_5_30",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=5)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=30)
        self.lines.sma_up_5_30 = bt.indicators.CrossUp(self.sma_short, self.sma_long)
        
class SMA_Crossdown_5_30(bt.Indicator):
    lines = ("sma_down_5_30",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=5)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=30)
        self.lines.sma_down_5_30 = bt.indicators.CrossDown(self.sma_short, self.sma_long)

class EMA_Crossup_12_26(bt.Indicator):
    lines = ("ema_up_12_26",)
    def __init__(self):
        self.ema_short = bt.indicators.ExponentialMovingAverage(self.data,period=12)
        self.ema_long = bt.indicators.ExponentialMovingAverage(self.data,period=26)
        self.lines.ema_up_12_26 = bt.indicators.CrossUp(self.ema_short, self.ema_long)
class EMA_Crossdown_12_26(bt.Indicator):
    lines = ("ema_down_12_26",)
    def __init__(self):
        self.ema_short = bt.indicators.ExponentialMovingAverage(self.data,period=12)
        self.ema_long = bt.indicators.ExponentialMovingAverage(self.data,period=26)
        self.lines.ema_down_12_26 = bt.indicators.CrossDown(self.ema_short, self.ema_long)
class MACD_Crossup(bt.Indicator):
    lines = ("macd_up",)
    def __init__(self):
        self.macd=bt.indicators.MACD(self.data)
        self.lines.macd_up=bt.indicators.CrossUp(self.macd.macd,self.macd.signal)

class MACD_Crossdown(bt.Indicator):
    lines = ("macd_down",)
    def __init__(self):
        self.macd=bt.indicators.MACD(self.data)
        self.lines.macd_down=bt.indicators.CrossDown(self.macd.macd,self.macd.signal)
class DEMA_CrossUp_21_55(bt.Indicator):
    lines = ("dema_up_21_55",)
    def __init__(self):
        self.dema_short = bt.indicators.DoubleExponentialMovingAverage(self.data,period=21)
        self.dema_long = bt.indicators.DoubleExponentialMovingAverage(self.data,period=55)
        self.lines.dema_up_21_55 = bt.indicators.CrossUp(self.dema_short, self.dema_long)
class DEMA_CrossDown_21_55(bt.Indicator):
    lines = ("dema_down_21_55",)
    def __init__(self):
        self.dema_short = bt.indicators.DoubleExponentialMovingAverage(self.data,period=21)
        self.dema_long = bt.indicators.DoubleExponentialMovingAverage(self.data,period=55)
        self.lines.dema_down_21_55 = bt.indicators.CrossDown(self.dema_short, self.dema_long)
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        self.sma_short_5=bt.indicators.SimpleMovingAverage(self.stock_px, period=5)
        self.sma_long_30=bt.indicators.SimpleMovingAverage(self.stock_px, period=30)
        self.sma_short_50 = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
        self.sma_long_200 = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
        self.ema_short=bt.indicators.ExponentialMovingAverage(self.stock_px,period=12)
        self.ema_long=bt.indicators.ExponentialMovingAverage(self.stock_px,period=26)
        self.macd=bt.indicators.MACD(self.stock_px)
        self.dema_short=bt.indicators.DoubleExponentialMovingAverage(self.stock_px,period=21)
        self.dema_long=bt.indicators.DoubleExponentialMovingAverage(self.stock_px,period=55)
        self.hma_30=bt.indicators.HullMA(self.stock_px,period=30)
        
        self.sma_up_50_200=SMA_Crossup_50_200(self.stock_px)
        self.sma_down_50_200=SMA_Crossdown_50_200(self.stock_px)
        self.sma_up_5_30=SMA_Crossup_5_30(self.stock_px)
        self.sma_down_5_30=SMA_Crossdown_5_30(self.stock_px)
        self.ema_up_12_26=EMA_Crossup_12_26(self.stock_px)
        self.ema_down_12_26=EMA_Crossdown_12_26(self.stock_px)
        self.macd_up=MACD_Crossup(self.stock_px)
        self.macd_down=MACD_Crossdown(self.stock_px)
        self.dema_up_21_55=DEMA_CrossUp_21_55(self.stock_px)
        self.dema_down_21_55=DEMA_CrossDown_21_55(self.stock_px)
        
#         self.sma_down_50_200=SMA_50_200_Down(self.stock_px)
#         self.sma_up_50_200=SMA_50_200_Up(self.stock_px)
        #self.psar=bt.indicators.PSAR(self.stock_px,period=2)
        #self.adx=bt.indicators.ADX(self.stock_px,period=14)
        
        #self.dsma = self.sma_short - self.sma_long
        #self.mom = bt.Cmp(self.sma_short, self.sma_long)
        #self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        #self.rsi_oversold = RSI_Oversold(self.stock_px)
        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30

        #self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        #self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size
        
        # Check if we are currently in a trade
        if(self.flag==0):
            #If not has our indicator triggered?
            if (self.sma_up_50_200[0] and not self.sma_up_50_200[-1]):
                print("SMA_up indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag=1
        elif(int(self.flag)==1):
            if (self.days_in_trade >= self.p.max_days_in_trade):
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                self.flag=0
            else:
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
           
                

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/AAPL_new.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='AAPL_STAT.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-04, Close, 30.57; Vol, 20.07
SMA_up indicator triggered...entering position
2010-01-05, Close, 30.63; Vol, 19.46
In position 2 days
2010-01-06, Close, 30.14; Vol, 19.81
In position 3 days
2010-01-07, Close, 30.08; Vol, 19.77
In position 4 days
2010-01-08, Close, 30.28; Vol, 19.92
In position 5 days
2010-01-11, Close, 30.02; Vol, 18.77
In position 6 days
2010-01-12, Close, 29.67; Vol, 18.67
In position 7 days
2010-01-13, Close, 30.09; Vol, 19.64
Max days to hold trade reached...closing position
2010-01-14, Close, 29.92; Vol, 18.69
2010-01-15, Close, 29.42; Vol, 19.51
2010-01-19, Close, 30.72; Vol, 21.50
2010-01-20, Close, 30.25; Vol, 22.69
2010-01-21, Close, 29.72; Vol, 23.22
2010-01-22, Close, 28.25; Vol, 25.74
2010-01-25, Close, 29.01; Vol, 28.31
2010-01-26, Close, 29.42; Vol, 32.33
2010-01-27, Close, 29.70; Vol, 33.60
2010-01-28, Close, 28.47; Vol, 33.78
2010-01-29, Close, 27.44; Vol, 35.66
2010-02-01, Close, 27.82; Vol, 35.21
2010-02-02, C